In [20]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
 #Drop the non-beneficial ID columns,'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [22]:
# Determine the number of unique values in each column.
unique_df=application_df.nunique()

# Display the unique value counts
print(unique_df)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [23]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [24]:
# Choose a cutoff value
cutoff_value = 200

# Create a list of application types to be replaced
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() <= cutoff_value].index.tolist()

# Replace in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check the updated value counts
print(application_df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [25]:
print(application_df.dtypes)

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object


In [26]:
# Look at CLASSIFICATION value counts for binning

classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the value counts
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [27]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_filtered = classification_counts[classification_counts > 1]

# Display the filtered value counts
print(classification_counts_filtered)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [28]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() <= cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [29]:
# Convert categorical data to numeric with `pd.get_dummies`

application_df=pd.get_dummies(application_df,dtype=float)


In [30]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [31]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Fit the scaler
#X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
# Define the deep learning model 
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=X_train_scaled.shape[1]))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5902 - accuracy: 0.7081
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5605 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5559 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5536 - accuracy: 0.7309
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7328
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7323
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7333
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7327
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5483 - accura

In [33]:
# Define the neural network model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")


Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5700 - accuracy: 0.7220
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7314
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7316
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7320
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7330
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7325
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7338
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7339
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5473 - accuracy: 0.7343
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7342

In [34]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scale the features using Min-Max Scaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network model
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile the model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model1.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model on the test data
loss, accuracy = model1.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")


Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5693 - accuracy: 0.7239
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5541 - accuracy: 0.7283
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5522 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5494 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7335
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5445 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accura

In [ ]:
#Save and export your results to an HDF5 file
model.save('AlphabetSoupCharity_Optimization.h5')

**Report** **on** **Performance** **of** **Deep** **Learning** **Model** ***for*** **Alphabet** **Soup** **Overview** **of** **the** **Analysis**:   

This analysis aims to develop a deep learning model using neural networks to classify whether or not the nonprofit foundation Alphabet Soup will be successful based on various features. The model aims to achieve high accuracy in predicting the success of a charity, which can assist Alphabet Soup in making informed decisions about funding allocation.

**Results:**

**Data Preprocessing:**

Target Variable: The target variable for the model is the "IS_SUCCESSFUL" column, which indicates whether a charity is successful or not.

Features: All columns except "IS_SUCCESSFUL," "EIN," and "NAME" are considered features for the model.

Variables to Remove: The "EIN" and "NAME" columns are removed from the input data as they are neither targets nor features and do not contribute to the prediction.

**Compiling, Training, and Evaluating the Model:**

Neurons, Layers, and Activation Functions: 

The selected neural network model consists of 128 neurons in the first hidden layer, 64 in the second hidden layer, and 32 in the third hidden layer. The activation function used in all hidden layers is the rectified linear unit (ReLU), effectively capturing non-linear patterns. In addition, the output layer has one neuron with a sigmoid activation function for binary classification.

Target Model Performance: The target model performance is to achieve an accuracy higher than 75%.

**Model** **Performance** **Results**:

Loss: 0.5529006123542786, Accuracy: 0.7257142663002014

Loss: 0.5560992956161499, Accuracy: 0.723498523235321

Loss: 0.5512644052505493, Accuracy: 0.72967928647995

Loss: 0.5579925775527954, Accuracy: 0.728863000869751

**Summary:** 

The deep learning model achieved an accuracy ranging from 72.57% to 72.85% in predicting the success of charity organizations. The deep learning model performed reasonably well but fell short of the target accuracy of 75%. To improve the model's performance, we can:

1. Increase model complexity by adding more layers or neurons.
2. Experiment with different activation functions to find.
3. Perform more extensive data preprocessing, including feature engineering and selection techniques, to identify the most relevant features for the classification task.


By implementing these recommendations and refining the model, higher accuracy and better predictive power can be achieved in identifying successful charities.
.